In [11]:
import pandas as pd

In [12]:
df_daily = pd.read_csv('data/air.csv')
hospital_df = pd.read_csv('data/hosp_data.csv')

/var/folders/7q/w_k7x3ss7fsgflp_442s596r0000gn/T/ipykernel_20054/571365695.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_daily = pd.read_csv('data/air.csv')


In [13]:
df_daily['date_local'] = pd.to_datetime(df_daily['date_local'], errors='coerce')
hospital_df['week_start'] = pd.to_datetime(hospital_df['week_start'], errors='coerce')
df_daily['month'] = df_daily['date_local'].dt.to_period('M')

epa_monthly = (
    df_daily.groupby(['month', 'parameter'])
    .agg({'arithmetic_mean': 'mean'})
    .reset_index()
)
hospital_df['month'] = hospital_df['week_start'].dt.to_period('M')

hospital_monthly = (
    hospital_df.groupby(['month', 'respiratory_category'])
    .agg({'percent': 'mean'})
    .reset_index()
)


In [14]:
merged_monthly = hospital_monthly.merge(
    epa_monthly,
    on='month',
    how='inner'
)

merged_monthly.shape


(555, 5)

In [15]:
merged_monthly[['arithmetic_mean', 'percent']].corr()


,arithmetic_mean,percent
arithmetic_mean,1.000000,0.013556
percent,0.013556,1.000000


In [16]:
merged_monthly = merged_monthly.sort_values('month')

for lag in [1, 2, 3]:
    merged_monthly[f"pollution_lag_{lag}"] = (
        merged_monthly.groupby('respiratory_category')['arithmetic_mean']
        .shift(lag)
    )


In [17]:
import statsmodels.api as sm
# drop any rows with NaN or inf in X or y
X = merged_monthly[['arithmetic_mean', 'pollution_lag_1', 'pollution_lag_2']].copy()
X = sm.add_constant(X)
y = merged_monthly['percent']

# remove rows with missing data
data = pd.concat([X, y], axis=1).dropna()

X_clean = data[['const', 'arithmetic_mean', 'pollution_lag_1', 'pollution_lag_2']]
y_clean = data['percent']

model = sm.OLS(y_clean, X_clean).fit()
print(model.summary())



                            OLS Regression Results                            
Dep. Variable:                percent   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                   0.05585
Date:                Mon, 08 Dec 2025   Prob (F-statistic):              0.983
Time:                        21:25:54   Log-Likelihood:                 860.09
No. Observations:                 545   AIC:                            -1712.
Df Residuals:                     541   BIC:                            -1695.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0266      0.014     

In [25]:
import hashlib
import os
sha_file = "sha256/sha256merged.csv"
os.makedirs(os.path.dirname(sha_file), exist_ok=True)
csv_bytes = merged_monthly.to_csv(index=False).encode("utf-8")
hash_value = hashlib.sha256(csv_bytes).hexdigest()
with open(sha_file, "w") as f:
    f.write(hash_value)